In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import sys
print(sys.executable)

In [ ]:
!jupyter kernelspec list

In [ ]:
import glob
# import imageio as iio
import matplotlib.pyplot as plt 
import numpy as np
import os
import pandas as pd

In [ ]:
from skimage import color
from skimage import io
from skimage.transform import resize

In [ ]:
ROOT_FILE_PATH = "/Users/adminpc-macmini/Library/CloudStorage/GoogleDrive-jeremias.rueck@gmail.com/Meine Ablage/Google Drive - Code/NTHU/final_project/data"
AREAS = ["prf-visualrois", "floc-bodies", "floc-faces", "floc-places", "floc-words", "streams", "all-vertices"]
SUBJ_LIST = ["subj01", "subj02", "subj03", "subj04", "subj05", "subj06", "subj07", "subj08"]
MASK_KEYS = ["lh.fsaverage_space", "rh.fsaverage_space", "lh.space", "rh.space", "mapping"]

In [ ]:
def ReadNpyFile(dir_path):
  return_dic = {}
  for path in glob.glob(dir_path + "/*.npy"):
    filename = os.path.splitext(path)[0].split("/")[-1]
    return_dic[filename] = np.load(path, allow_pickle = True)
  return return_dic

def ReadFmriFile(dir_path):
  return_dic = {}
  for path in glob.glob(dir_path + "/*.npy"):
    filename = os.path.splitext(path)[0].split("/")[-1]
    return_dic[filename[0]] = np.load(path, allow_pickle = True)
  return return_dic

def ReadMaskFile(dir_path):
  return_dic = {area: {} for area in AREAS}
  for path in glob.glob(dir_path + "/*.npy"):
    filename = os.path.splitext(path)[0].split("/")[-1]
    for area in AREAS:
      if area in filename:
        filename = "".join(i.strip("_") for i in filename.split(area))
        return_dic[area][filename] = np.load(path, allow_pickle = True) if filename[0] != "m" else np.load(path, allow_pickle = True).item()
        break
  return return_dic

def FetchPngFile(dir_path, index, mode = "rgb"):
  # img = iio.v3.imread()
  img = io.imread(os.path.join(dir_path, f'{index}.png'))
  if mode == "rgb":
    return img
  if mode == "hsv":
    return color.rgb2hsv(img)
    # return matplotlib.colors.rgb_to_hsv(img)
  if mode == "gray":
    return color.rgb2gray(img)
    # return np.dot(img, [0.299, 0.587, 0.114])

In [ ]:
def train_val_split(x, y, ratio, random_state=42):
  
  r = np.random.RandomState(random_state)
  
  idx = r.permutation(x.index)
  x_per = x.reindex(idx)
  y_per = y.reindex(idx)

  train_size = int(x.shape[0] * ratio)
  
  x_train = x_per[0:train_size]
  y_train = y_per[0:train_size]
  
  x_val = x_per[train_size:]
  y_val = y_per[train_size:]
  
  print('Training_dimension: ' + str(x_train.shape) + ' | ' + str(y_train.shape))
  print('Validation_dimension: ' + str(x_val.shape) + ' | ' + str(y_val.shape))

  return x_train, x_val, y_train, y_val

In [ ]:
class ReadObject:
  
  fmri = None           # dict(string -> ndarray, float32)
  masks = None          # dict(string "AREA" -> (dict(int -> string), string "HEMISPHERE, " -> ndarray))
  
  image_labels = None   # dataframe
  label_names = None    # list

  subject = None        # string
  
  def __init__(self, subject):
    self.subject = subject

  # Mode = "train" | "test"
  def load(self, mode = "train"):
    
    self.image_labels = pd.read_csv(os.path.join(ROOT_FILE_PATH, "image_infos", f"{self.subject}_infos_train.csv"))
    self.image_labels = self.image_labels.drop(self.image_labels.columns[[0, 1]], axis=1)
    self.label_names = self.image_labels.columns
    
    self.image_labels = self.image_labels.to_numpy(dtype=int)

    print('Images: ' + str(self.image_labels.shape[0]))
    print('Labels: ' + str(self.image_labels.shape[1]))
    
    self.masks = ReadMaskFile(os.path.join(ROOT_FILE_PATH, self.subject , "roi_masks"))
    print('Masks: ' + str(len(self.masks.keys())))
        
    if mode == "train":
      self.fmri = ReadFmriFile(os.path.join(ROOT_FILE_PATH, self.subject, "training_split", "training_fmri"))
    elif mode == "test":
      self.fmri = ReadFmriFile(os.path.join(ROOT_FILE_PATH, self.subject, "test_split", "test_fmri"))

  def split_data(self, count=None, ratio=None, permutate=True, image_mode = "rgb", random_state=42):
    
    if ratio is None:
      ratio = 1
    
    random_state = np.random.RandomState(random_state)
    
    if count is not None and count > 0:
      fmri_l_copy = self.fmri['l'][:count]
      fmri_r_copy = self.fmri['r'][:count]
      labels_copy = self.image_labels[:count]
    else:
      fmri_l_copy = self.fmri['l']
      fmri_r_copy = self.fmri['r']
      labels_copy = self.image_labels
    
    loaded_images = np.stack([FetchPngFile(os.path.join(ROOT_FILE_PATH, self.subject,
                      "training_split", "training_images"), i, mode = image_mode) for i in range(labels_copy.shape[0])])
    
    if permutate:
      
        idx = random_state.permutation(labels_copy.shape[0])
        
        fmri_l_copy = fmri_l_copy[idx]
        fmri_r_copy = fmri_r_copy[idx]
        labels_copy = labels_copy[idx]
        loaded_images = loaded_images[idx]
        
    train_size = int(labels_copy.shape[0] * ratio)
  
    fmri_l_train = fmri_l_copy[0:train_size]
    fmri_r_train = fmri_r_copy[0:train_size]
    labels_train = labels_copy[0:train_size]
    images_train = loaded_images[0:train_size]
    
    fmri_l_val = fmri_l_copy[train_size:]
    fmri_r_val = fmri_r_copy[train_size:]
    labels_val = labels_copy[train_size:]
    images_val = loaded_images[train_size:]
    
    return (labels_train, labels_val), (fmri_l_train, fmri_l_val), (fmri_r_train, fmri_r_val), (images_train, images_val)

In [ ]:
%matplotlib inline
def test_data(label_names, labels_list, image, fmri_l, fmri_r):
  labels = np.array(label_names)[np.where(labels_list != 0)] 
  print(labels)
  plt.imshow(image)
  plt.show()

In [ ]:
obj = ReadObject(SUBJ_LIST[0])
obj.load()

In [ ]:
(
  (labels_train, labels_val), (fmri_l_train, fmri_l_val),
  (fmri_r_train, fmri_r_val), (images_train, images_val)
) = (obj.split_data(count=5000, ratio=0.995, image_mode= "rgb"))

In [ ]:
print(labels_train.shape)
print(labels_val.shape)
print(fmri_l_train.shape)
print(fmri_l_val.shape)
print(fmri_r_train.shape)
print(fmri_r_val.shape)
print(images_train.shape)
print(images_val.shape)

In [ ]:
index = 183
test_data(obj.label_names, labels_train[index], images_train[index], None, None)

In [ ]:
print(images_train.shape)

In [ ]:
print(fmri_l_train.shape)
print(fmri_r_train.shape)
X_train = combined_array = np.concatenate((fmri_l_train, fmri_r_train), axis=1)
print(X_train.shape)

In [ ]:
print(np.max(X_train))
print(np.min(X_train))

In [ ]:
# Define the original range
old_min = np.min(X_train)
old_max = np.max(X_train)

# Define the new range you want to map to
new_min = 0
new_max = 1

# Map the values from the original range to the new range
mapped_X_train = (X_train - old_min) * (new_max - new_min) / (old_max - old_min) + new_min

print(np.min(mapped_X_train))
print(np.max(mapped_X_train))

In [ ]:
Y_train = labels_train
print(Y_train.shape)

In [ ]:
from tensorflow import keras

In [ ]:
model = keras.models.Sequential()

model.add(keras.layers.Conv1D(filters=32, kernel_size=(8,), activation=keras.layers.LeakyReLU(), input_shape=(39548, 1)))
model.add(keras.layers.Conv1D(filters=32, kernel_size=(8,), activation=keras.layers.LeakyReLU()))
model.add(keras.layers.MaxPool1D())

model.add(keras.layers.Conv1D(filters=64, kernel_size=(4,), activation=keras.layers.LeakyReLU()))
model.add(keras.layers.Conv1D(filters=64, kernel_size=(4,), activation=keras.layers.LeakyReLU()))
model.add(keras.layers.MaxPool1D())

model.add(keras.layers.Conv1D(filters=128, kernel_size=(3,), activation=keras.layers.LeakyReLU()))
model.add(keras.layers.Conv1D(filters=128, kernel_size=(3,), activation=keras.layers.LeakyReLU()))
model.add(keras.layers.MaxPool1D())

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(512, activation="relu"))
model.add(keras.layers.Dense(256, activation="relu"))
model.add(keras.layers.Dense(133, activation="softmax"))

model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=["accuracy"]) # keras.optimizers.Adam(learning_rate=0.000001)
model.summary()

In [ ]:
model.fit(X_train.reshape(-1, mapped_X_train.shape[1], 1), Y_train, batch_size=32, epochs=10)

In [ ]:
# Fill with zeros till 40000
target_size = 40000
additional_rows = np.zeros(shape=(4975, target_size - X_train.shape[1]))
print(additional_rows.shape)
expanded_40000_X_train = np.concatenate((X_train, additional_rows), axis=1)
print(expanded_40000_X_train.shape)

In [ ]:
count = 500
resized_300_Y_train = np.zeros((count, 300, 300, 3))
for index in range(count):
  print(index)
  resized_300_Y_train[index] = resize(images_train[index], (300, 300, 3), anti_aliasing=True)
print(resized_300_Y_train.shape)

In [ ]:
encoder1 = keras.models.Sequential()

encoder1.add(keras.layers.Conv2D(1, kernel_size=(3, 3), padding="same", activation="relu", input_shape=(200, 200, 1)))

encoder1.add(keras.layers.Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu"))
encoder1.add(keras.layers.Conv2D(32, kernel_size=(3, 3), padding="same", activation="sigmoid"))
encoder1.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

encoder1.add(keras.layers.Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu"))
encoder1.add(keras.layers.Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu"))
encoder1.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

encoder1.add(keras.layers.Conv2D(128, kernel_size=(3, 3), padding="same", activation="relu"))
encoder1.add(keras.layers.Conv2D(128, kernel_size=(3, 3), padding="same", activation="relu"))
encoder1.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
encoder1.summary()

decoder1 = keras.models.Sequential()

decoder1.add(keras.layers.Conv2D(128, kernel_size=(3, 3), padding="same", activation="relu", input_shape=(25, 25, 128)))
decoder1.add(keras.layers.Conv2D(128, kernel_size=(3, 3), padding="same", activation="sigmoid"))
decoder1.add(keras.layers.UpSampling2D(size=(3, 3)))

decoder1.add(keras.layers.Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu"))
decoder1.add(keras.layers.Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu"))
decoder1.add(keras.layers.UpSampling2D(size=(2, 2)))

decoder1.add(keras.layers.Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu"))
decoder1.add(keras.layers.Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu"))
decoder1.add(keras.layers.UpSampling2D(size=(2, 2)))

decoder1.add(keras.layers.Conv2D(3, kernel_size=(3, 3), padding="same", activation="sigmoid"))
decoder1.summary()

In [ ]:
model1 = keras.models.Sequential()
model1.add(encoder1)
model1.add(decoder1)

model1.compile(optimizer="sgd", loss="mse") # "adam", "rmsprop"

In [ ]:
model1.fit(expanded_40000_X_train.reshape(-1, 200, 200)[:500], resized_300_Y_train, batch_size=32, epochs=10)

In [ ]:
count = 500
resized_200_Y_train = np.zeros((count, 200, 200, 3))
for index in range(count):
  print(index)
  resized_200_Y_train[index] = resize(images_train[index], (200, 200, 3), anti_aliasing=True)
print(resized_200_Y_train.shape)

In [ ]:
encoder2 = keras.models.Sequential()

encoder2.add(keras.layers.Conv2D(32, kernel_size=4, strides=2, input_shape=(200, 200, 1)))
encoder2.add(keras.layers.BatchNormalization())
encoder2.add(keras.layers.LeakyReLU())
encoder2.add(keras.layers.Dropout(0.1))

encoder2.add(keras.layers.Conv2D(64, kernel_size=3, strides=2))
encoder2.add(keras.layers.BatchNormalization())
encoder2.add(keras.layers.LeakyReLU())
encoder2.add(keras.layers.Dropout(0.1))

encoder2.summary()

bridge2 = keras.models.Sequential()

bridge2.add(keras.layers.Conv2D(128, kernel_size=3, strides=2, input_shape=(49, 49, 64)))

bridge2.summary()

decoder2 = keras.models.Sequential()

decoder2.add(keras.layers.Dropout(0.1, input_shape=(24, 24, 128)))
decoder2.add(keras.layers.LeakyReLU())
decoder2.add(keras.layers.BatchNormalization())
decoder2.add(keras.layers.Conv2DTranspose(64, kernel_size=3, strides=2))
decoder2.add(keras.layers.BatchNormalization())
decoder2.add(keras.layers.LeakyReLU())
decoder2.add(keras.layers.Dropout(0.1))

decoder2.add(keras.layers.Conv2DTranspose(32, kernel_size=3, strides=2))
decoder2.add(keras.layers.BatchNormalization())
decoder2.add(keras.layers.LeakyReLU())
decoder2.add(keras.layers.Dropout(0.1))

decoder2.add(keras.layers.Conv2DTranspose(3, kernel_size=4, strides=2))

decoder2.summary()

In [ ]:
model2 = keras.models.Sequential()

model2.add(encoder2)
model2.add(bridge2)
model2.add(decoder2)

model2.summary()
model2.compile(optimizer="sgd", loss="mse")

In [ ]:
model2.fit(expanded_40000_X_train.reshape(-1, 200, 200)[:500], resized_200_Y_train, batch_size=32, epochs=10)